In [30]:
import requests
import pandas as pd
from time import sleep
class CkanConsumer:
    def __init__(self, main_url:str, secure:bool = True):
        if secure:
            self.url = f'https://{main_url}/api/action/datastore_search'
        else:
            self.url = f'http://{main_url}/api/action/datastore_search'

    def request(self, resource_id) -> pd.DataFrame:
        params = {
            'limit': 1000,
            'offset': 0,
            'resource_id': resource_id
        }
        data = []

        print(f'[INFO] - Getting data from {self.url} on resource {resource_id} at offset {len(data)}')
        response = requests.get(self.url, params=params)
        total = response.json()['result']['total']
        data.extend(response.json()['result']['records'])

        while len(data) < total:
            params['offset'] = len(data)
            print(f'[INFO] - Getting data from {self.url} on resource {resource_id} at offset {len(data)}')
            response = requests.get(self.url, params=params)
            while response.status_code != 200:
                print(f'[ERROR] - Response code {response.status_code} from {self.url} on resource {resource_id} at offset {len(data)}')
                print(f'[INFO] - Waiting 5 seconds before trying again')
                sleep(5)
                response = requests.get(self.url, params=params)
            data.extend(response.json()['result']['records'])
        
        print(f'[INFO] - Total of {len(data)} records retrieved from {self.url} on resource {resource_id}')
        #Reassembling the dataframe
        df = pd.DataFrame(data)
        return df

In [32]:
# Use ckan_consumer.request(resource_id) to get a dataframe from a resource_id
ufrn_consumer = CkanConsumer('dados.ufrn.br')
cursos_ufrn = ufrn_consumer.request('a10bc434-9a2d-491a-ae8c-41cf643c35bc')
docentes_ufrn = ufrn_consumer.request('6a8e5461-e748-45c6-aac6-432188d88dde')
discentes_ufrn = ufrn_consumer.request('14afbb6c-395e-411c-b24d-0e494cb95866')

[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource a10bc434-9a2d-491a-ae8c-41cf643c35bc at offset 0
[INFO] - Total of 120 records retrieved from https://dados.ufrn.br/api/action/datastore_search on resource a10bc434-9a2d-491a-ae8c-41cf643c35bc
[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource 6a8e5461-e748-45c6-aac6-432188d88dde at offset 0
[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource 6a8e5461-e748-45c6-aac6-432188d88dde at offset 1000
[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource 6a8e5461-e748-45c6-aac6-432188d88dde at offset 2000
[INFO] - Total of 2772 records retrieved from https://dados.ufrn.br/api/action/datastore_search on resource 6a8e5461-e748-45c6-aac6-432188d88dde
[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource 14afbb6c-395e-411c-b24d-0e494cb95866 at offset 0
[INFO] - Gettin

In [41]:
cursos_ufrn.head()

,id_unidade_responsavel,area_conhecimento,unidade_responsavel,id_coordenador,codigo_inep,campus,nivel_ensino,nome,portaria_reconhecimento,id_curso,...,tipo_ciclo_formacao,convenio_academico,modalidade_educacao,municipio,situacao_curso,data_funcionamento,coordenador,turno,_id,dou
0,31011,Ciências Sociais Aplicadas,"FACULDADE DE ENGENHARIA, LETRAS E CIÊNCIAS SOC...",5752389,12355,Campus Currais Novos,GRADUAÇÃO,ADMINISTRAÇÃO,Portaria nº 272/2017 - MEC,2000004,...,Um ciclo,,Presencial,CURRAIS NOVOS,ATIVO,1978-09-15T00:00:00,MARILENE BIZERRA DA COSTA,Vespertino e Noturno,1,2017-04-04T00:00:00
1,443,Ciências Sociais Aplicadas,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS,24344,12311,Campus Central,GRADUAÇÃO,ADMINISTRAÇÃO,Portaria nº 272/2017 - MEC,2000002,...,Um ciclo,,Presencial,NATAL,ATIVO,1971-09-16T00:00:00,CARLOS ALBERTO FREIRE MEDEIROS,Matutino e Noturno,2,2017-04-04T00:00:00
2,443,Ciências Sociais Aplicadas,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS,5756793,1563343,,GRADUAÇÃO,ADMINISTRAÇÃO PÚBLICA,,149654895,...,Um ciclo,,Presencial,NATAL,ATIVO,None,FABIO RESENDE DE ARAUJO,,3,None
3,443,Ciências Sociais Aplicadas,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS,5753587,1151028,,GRADUAÇÃO,ADMINISTRAÇÃO PÚBLICA,Portaria nº 226/2014 - DIREG/MEC,15315770,...,Um ciclo,,A Distância,NATAL,ATIVO,None,CARLOS DAVID CEQUEIRA FEITOR,,4,2014-04-11T00:00:00
4,351,Outra,ESCOLA AGRÍCOLA DE JUNDIAÍ,5760230,1188193,,GRADUAÇÃO,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS,Portaria MEC/SERES Nº 922/2018,94598200,...,Um ciclo,,Presencial,MACAÍBA,ATIVO,None,CARLA DA COSTA FERNANDES CURVELO,,5,2018-12-28T00:00:00


In [34]:
from hashlib import md5
treat = discentes_ufrn.apply(lambda df: pd.Series({
    'nome': str(df['nome_discente']).upper(),
    'id': md5((str(df['matricula'])+'ufrn'+'14afbb6c-395e-411c-b24d-0e494cb95866').encode()).hexdigest(),
    'codigo': str(df['id_curso']),
    'sexo': df['sexo'],
    'data_ingresso': f"{str(df['ano_ingresso'])}.{str(df['periodo_ingresso'])}",
    'curso': df['nome_curso']
}), axis=1)


In [38]:
#do the above cell like a function
def mapper_cursos(
        dataframe: pd.DataFrame,
        nome_curso: str,
        id_curso: str,
        instituto: str,
        resource_id: str
    ) -> pd.DataFrame:
    result = dataframe.apply(lambda df: pd.Series({
        'nome': str(df[nome_curso]).upper(),
        'id': md5((f"{df[nome_curso]}{instituto}{resource_id}").encode()).hexdigest(),
        'codigo': str(df[id_curso]),
        'instituicao': instituto
    }), axis = 1)
    return result

df = mapper_discentes(discentes_ufrn, 'nome_discente', 'matricula', 'sexo', 'ano_ingresso', 'periodo_ingresso', 'nome_curso', 'ufrn', '14afbb6c-395e-411c-b24d-0e494cb95866')

In [39]:
df.head()

,nome,id,matricula,sexo,data_ingresso,curso
0,AARON VINICIUS MAIA SOBRINHO,6f91a90b48840e78a5b05a5bdcc39283,20220017174,M,2022.1,ENGENHARIA DE PETRÓLEO
1,ABÃ BARBOSA DA SILVA,067ebfd1e4d331f9e707e343462954df,20220057240,M,2022.2,ENGENHARIA MECÂNICA
2,ABDA BEATRIZ DE ARAUJO PINHEIRO,51bdd617924f6b3c8be29838786fba55,20220046390,F,2022.1,BACHARELADO INTERDISCIPLINAR EM HUMANIDADES
3,ABDALLAH SAID SALEH YUSUF,1461a3cb16dd3a16ab9d378f921cc8fa,20220077682,M,2022.2,
4,ABEL BARBOSA DE SOUZA NETO,c02153fcda7e2cfa08087d1600e66c5a,20223009354,M,2022.1,CURSO TÉCNICO DA METRÓPOLE DIGITAL
